In [ ]:
#!git clone https://github.com/shredder67/svd-lorafa.git
#!git clone https://github.com/nyu-mll/GLUE-baselines.git
#%cd GLUE-baselines




In [ ]:
#!python download_glue_data.py

In [ ]:
#%mkdir ../svd-lorafa/data

In [ ]:
#%cp -r ../GLUE-baselines/glue_data/STS-B ../svd-lorafa/data/STS-B
#%cd ./svd-lorafa

/content/svd-lorafa


In [ ]:
import csv
import time
import json
import warnings
from functools import partial
warnings.filterwarnings('always')

import torch
import numpy as np
import pandas as pd
torch.manual_seed(0)
from torch import nn
import matplotlib.pyplot as plt
from tqdm.notebook import trange, tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaModel
from lora import *
from datasets import GLUEDatasetRoberta

Выходные данные были обрезаны до нескольких последних строк (5000).
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.

In [ ]:
RobertaModel.from_pretrained('roberta-base')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bo

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropou

We will follow LoRA-FA experiment setting, where in paper they state that they apply lora to all 4 linear layers in MultiHeadAttention (see section 2.2): 3 layers that produce $Q$, $K$ and $V$ and also final linear layer weights

<center>
<img src="imgs/mha.png" width=300 height=400/>
</center>

In [ ]:
def construct_lorafa_config(model, rank, init_method='svd'):
    """Each layer has a unique name within module hierarchy, so we can identify
    them for lora parametrization"""
    config = {}
    for name, module in model.named_modules():
        if isinstance(module, nn.Linear) and any([name.split('.')[-1] == n for n in ['dense', 'query', 'key', 'value',]]) and '.attention.' in name:
            config[name] = {
                nn.Linear: {
                    "weight": partial(
                        LoRAFAParametrization.from_linear,
                        rank=rank,
                        init_method=init_method, # set svd as initiazliation method
                        original_weights=module.weight # pass weights for svd init
                    ),
                }
            }
    return config

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Fine-tuning stage

## STS-B

### Prepare model

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')
model.pooler.activation = nn.Linear(768, 1)
model.pooler.dense.out_features = 1

# example
text1 = ["Simple text", "Cat", "House"]
text2 = ["Hard text", "Dog", "home"]

encoded_pair = tokenizer(
    text1,
    text2,
    padding='max_length',
    truncation=True,
    max_length=128,  # Maximum sequence length, adjust as needed
    return_tensors='pt'
)

output = model(**encoded_pair)
output.pooler_output.flatten()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bo

tensor([0.1012, 0.1006, 0.0982], grad_fn=<ViewBackward0>)

In [ ]:
def freeze_nonlora(model):
    for name, parameters in model.named_parameters():
        if not name_is_lora(name) and not 'pooler' in name: # don't want to freeze last original layer too
            parameters.requires_grad = False

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def get_hot_parameters(model):
    for _, params in model.named_parameters():
            if params.requires_grad:
                 yield params

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
rank = 2
lora_roberta_config = construct_lorafa_config(model, rank=rank, init_method='kaiming') # kaiming, svd
# list(lora_roberta_config.keys())
add_lora_by_layer_names(model, lora_roberta_config)
freeze_nonlora(model)

model = model.to(device)

parameters = [
    {"params": list(get_hot_parameters(model))},
]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bo

In [ ]:
#parameters[0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Prepare data

In [ ]:
data_dir = 'data/STS-B'
train_file = data_dir + '/train.tsv'
test_file = data_dir + '/dev.tsv'

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
train_dataset = GLUEDatasetRoberta(train_file, tokenizer, benchmark='stsb')
test_dataset = GLUEDatasetRoberta(test_file, tokenizer, benchmark='stsb')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bo

In [ ]:
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size,)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Train

In [ ]:
n_epoch = 80
warmup_ratio = 0.06
n_steps = len(train_loader) * n_epoch
warmup_steps = warmup_ratio * n_steps

def lr_lambda(current_step):
    if current_step <= warmup_steps:
        return (current_step + 1) / max(1, warmup_steps)
    else:
        return (n_steps - current_step) / (max(1, n_steps - warmup_steps))

optimizer = torch.optim.AdamW(parameters, lr=4e-4)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_lambda)
criteria = nn.MSELoss()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#train_loader

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
from scipy.stats import pearsonr

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bo

In [ ]:
#parameters.keys().list()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#parameters[0]['params']

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
parameters[0]['params'][0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Parameter containing:
tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]], device='cuda:0', requires_grad=True)

In [ ]:
#label

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
for input_ids, attention_mask, label in train_loader: # training
    label = label.float()
    if torch.isnan(label[0]):
        print('error')
    #print(label)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


error


In [ ]:
for input_ids, attention_mask, label in train_loader: # training
    label = label.float()
    for i in range (label.shape[0]):
        if torch.isnan(label[i]):
            print('error_label')
            print(input_ids)
            break

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


error_label
tensor([[    0,   133,  1275,  ...,     1,     1,     1],
        [    0,   133,  7161,  ...,     1,     1,     1],
        [    0,   133,  2270,  ...,     1,     1,     1],
        ...,
        [    0, 43394,  2784,  ...,     1,     1,     1],
        [    0, 25216,    34,  ...,     1,     1,     1],
        [    0, 38011,    12,  ...,     1,     1,     1]])
error_label
tensor([[    0,   250, 29874,  ...,     1,     1,     1],
        [    0, 36118,  1891,  ...,     1,     1,     1],
        [    0,   250,   693,  ...,     1,     1,     1],
        ...,
        [    0,   250,   313,  ...,     1,     1,     1],
        [    0,  9058,  3678,  ...,     1,     1,     1],
        [    0, 37779,   118,  ...,     1,     1,     1]])
error_label
tensor([[    0,  4148,     5,  ...,     1,     1,     1],
        [    0,   250,  2143,  ...,     1,     1,     1],
        [    0,   717,  4308,  ...,     1,     1,     1],
        ...,
        [    0,   102,   333,  ...,     1,     1,    

In [ ]:
pbar = tqdm(total=n_steps, desc='Training', position=0)
pbar_test = tqdm(total=len(test_loader), desc='Validating', position=1)

training_history= {'train_loss': [],
                   'val_loss': [],
                   'val_pearson': [],}

for epoch in range(n_epoch):

    step_loss = []
    model.train()
    for input_ids, attention_mask, label in train_loader: # training
        label = label.float()
        if torch.isnan(label[0]):
            print('error_label')
            break
        #label = label.type(torch.LongTensor)   # casting to long
        #print("hi")
        output = model(input_ids.to(device), attention_mask.to(device))
        # Ensure the logits are in the right data type (float)
        #logits_float = output.logits.float()

        # Ensure the targets/labels are in the right data type (long)
        #label_long = label.long()
        #label = label.type(torch.LongTensor)
        pooler_outs = output.pooler_output.flatten()
        if torch.isnan(pooler_outs[0]):
            print('error_pooler')
            break
        loss = criteria(pooler_outs, label.to(device))

        optimizer.zero_grad()
        loss.backward()
        print(parameters[0]['params'][0])
        torch.nn.utils.clip_grad_norm_(parameters[0]['params'], 1)
        print(parameters[0]['params'][0])
        optimizer.step()
        scheduler.step()

        step_loss.append(loss.item())

        pbar.update()
    train_loss = np.mean(step_loss)

# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> VALIDATION AND PROGRESS BARS <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< #

    if (epoch + 1) % 5 == 0 or epoch > 0.6 * n_epoch: # validation
        if epoch != 4:
            pbar_test.update(-pbar_test.total) # for refreshing pbars
        step_loss = []
        all_labels = []
        all_predictions = []
        model.eval()
        for input_ids, attention_mask, label in test_loader:
            #label = label.type(torch.LongTensor)   # casting to long
            output = model(input_ids.to(device), attention_mask.to(device))
            loss = criteria(output.pooler_output.flatten(), label.to(device))
            step_loss.append(loss.item())

            # Collect all labels and predictions
            all_labels.extend(label.detach().cpu().numpy())
            all_predictions.extend(output.pooler_output.detach().cpu().numpy().flatten())  # Assuming single output per instance

            pbar_test.update()

        # Instead of accuracy, compute Pearson correlation
        val_pearson, _ = pearsonr(all_predictions, all_labels)

        val_loss = np.mean(step_loss)
        training_history['val_loss'].append(val_loss)
        training_history['val_pearson'].append(val_pearson)

        pbar_test.set_postfix({'val_loss': val_loss,
                               'val_pearson': val_pearson,})

    pbar.set_postfix({'train_loss': train_loss})
    training_history['train_loss'].append(train_loss)

pbar.close()
pbar_test.close()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bo

Training:   0%|          | 0/7200 [00:00<?, ?it/s]

Validating:   0%|          | 0/23 [00:00<?, ?it/s]

Parameter containing:
tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]], device='cuda:0', requires_grad=True)
error_pooler
error_pooler


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


...

error_pooler


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


error_pooler


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


OutOfMemoryError: ignored

In [ ]:
output.pooler_output

In [ ]:
training_history

In [ ]:
torch.isnan(parameters[0]['params'][0])

In [ ]:
with open('history.kaiming.json', 'w') as f:
    json.dump(training_history, f)

In [ ]:
#%cd /kaggle/working

In [ ]:
#from IPython.display import FileLink

In [ ]:
#FileLink(r'history.kaiming.json')